# PE Halo

This notebook will exercise your learning of Serialbox and porting from Fortran to Python.

This particular example will have mixed-starting indices for the various arrays within the Fortran code.  We'll let you determine what you think are the appropriate variables to pass into Python in order for you to write a Python/GT4Py port of the following Fortran code.

## Fortran to port 

In [1]:
%%writefile edge_pressure.F90

subroutine compute_edge_pressure(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
    integer, intent(in) :: is, ie, js, je, isd, ied, jsd, jed, npz
    double precision, intent(in) :: ptop
    double precision, intent(in), dimension(isd:ied,jsd:jed,npz):: delp
    double precision, intent(inout), dimension(is-1:ie+1,npz+1,js-1:je+1):: pe
    integer:: i,j,k

    do j=js,je
     pe(is-1,1,j) = ptop
     pe(ie+1,1,j) = ptop
     do k=1,npz
        pe(is-1,k+1,j) = pe(is-1,k,j) + delp(is-1,j,k)
        pe(ie+1,k+1,j) = pe(ie+1,k,j) + delp(ie+1,j,k)
     enddo
    enddo
    
    do i=is-1,ie+1
     pe(i,1,js-1) = ptop
     pe(i,1,je+1) = ptop
     do k=1,npz
        pe(i,k+1,js-1) = pe(i,k,js-1) + delp(i,js-1,k)
        pe(i,k+1,je+1) = pe(i,k,je+1) + delp(i,je+1,k)
     enddo
  enddo
end subroutine

program edge_pressure

  implicit none
    
  integer :: is, ie, js, je, isd, ied, jsd, jed, npz
  double precision :: ptop
  double precision, allocatable, dimension(:,:,:) :: delp, pe
  
  ! Initialize Serialbox and serialize the data you deem important for you to port this code
  ! to Python/GT4Py

  is = 1
  ie = 48
  js = 1
  je = 48
  isd = -2
  ied = 51
  jsd = -2
  jed = 51
  npz = 79
  ptop = 300.0
  allocate(delp(isd:ied, jsd:jed, npz))
  allocate(pe(is-1:ie+1,npz+1,js-1:je+1))
  call compute_edge_pressure(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
end program

Overwriting edge_pressure.F90


Run the following cell to compile and run the above Fortran code

In [2]:
%%bash

[ -f serialize_edge_pressure ] && rm serialize_edge_pressure
[ -f s_edge_pressure.F90 ] && rm s_edge_pressure.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_edge_pressure.F90 edge_pressure.F90
gfortran -O3 -cpp -DSERIALIZE \
    -o serialize_edge_pressure s_edge_pressure.F90 \
    -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
    -lpthread -lstdc++ -lstdc++fs
rm -rf ./data_pe
./serialize_edge_pressure
ls -lh data_pe

Processing file edge_pressure.F90
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
total 4.9M
-rw-r--r-- 1 rgeorge noaa-hpc 1.1K Nov 13 11:27 ArchiveMetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc 8.3K Nov 13 11:27 MetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc 1.8M Nov 13 11:27 port_delp.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 13 11:27 port_nhalo.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 13 11:27 port_nx.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 13 11:27 port_ny.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 13 11:27 port_nz.dat
-rw-r--r-- 1 rgeorge noaa-hpc 3.1M Nov 13 11:27 port_pe.dat
-rw-r--r-- 1 rgeorge noaa-hpc    8 Nov 13 11:27 port_ptop.dat


## Python Solution A
Using array size that is the max shape of all data arrays, matching indices.

In [5]:
#!/usr/bin/env python3

import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser


Finished running comparison tests!


## Python Solution B
Using array size that is the minimum size required for this computation.

In [4]:
#!/usr/bin/env python3

import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser


Finished running comparison tests!
